In [1]:
import path

In [2]:
import glob

In [3]:
import os.path

In [4]:

import pandas as pd

In [5]:
import numpy as np
    

In [6]:
import os

In [7]:
import datetime

In [8]:
 pd.options.mode.chained_assignment = None  # default='warn'

In [9]:
#cap_survey = 'W:\\hs database\\survey.csv'

In [10]:
folder_path_survey =  r"W:\\hs database\\survey_here"

In [11]:
folder_path_hs_report = r"W:\\hs database\\hs_report_here"

In [12]:
file_type =r'\*csv'

In [13]:
hs_data_import = glob.glob(folder_path_hs_report + file_type)

In [14]:
cap_survey_import = glob.glob(folder_path_survey + file_type)

In [15]:
max_file_survey=max(cap_survey_import, key=os.path.getctime)

In [16]:
max_file_hs_report = max(hs_data_import, key=os.path.getctime)

In [17]:
cap_survey_df =pd.read_csv(max_file_survey)

In [18]:
hs_data_df = pd.read_csv(max_file_hs_report)

In [19]:
cap_survey_df['Date'] = cap_survey_df['Date'].str.replace(r'UTC', '') #removes UTC from the date cell so we can convert to standard date format

In [20]:
cap_survey_df['Date'] = pd.to_datetime(cap_survey_df['Date'], format='%Y-%m-%d') #formatting date column to date data

In [21]:
survey_filtered_df= cap_survey_df[(cap_survey_df['Date'] > "2022-02-01") & (cap_survey_df['Date'] < "2022-03 - 01")] #filtering to include the last month's responses

In [22]:
survey_filtered_df.dtypes

Name                                                                                                                                                      object
Date                                                                                                                                              datetime64[ns]
Email Address                                                                                                                                             object
Was your visit to Career Services long enough?                                                                                                            object
To what extent did you find your visit to Career Services to be of value?                                                                                 object
Indicate all of the resources or skills that you are more confident with because of your visit to Career Services.                                        object
How likely is it that you would re

In [23]:
#survey_filtered_df.head() #shows the top five results

In [24]:
survey_filtered_df.drop(survey_filtered_df.columns[3:6], axis=1, inplace = True) #removing unneeded columns from survey sheet

In [25]:
#survey_filtered_df.head()

In [26]:
#survey_filtered_df['Date'] = pd.to_datetime(survey_filtered_df['Date'], format='%Y-%m-%d') #format new df to date

In [27]:
hs_data_df = hs_data_df.rename(columns={"Appointments Start Date Date": "Date"}) #change column title to match other dataframe

In [28]:
hs_data_df =hs_data_df.rename(columns={"Student Email - Institution": "Email Address"}) #change column title to match dataframe

In [29]:
hs_data_df.dtypes

Email Address             object
Appointment Types Name    object
Student First Name        object
Student Last Name         object
Date                      object
Staff Member Last Name    object
dtype: object

In [30]:
hs_data_df['Date'] = pd.to_datetime(hs_data_df['Date'], format='%Y-%m-%d') #format column to date

In [31]:
hs_data_df.dtypes

Email Address                     object
Appointment Types Name            object
Student First Name                object
Student Last Name                 object
Date                      datetime64[ns]
Staff Member Last Name            object
dtype: object

In [32]:
mask_hs_data_df = hs_data_df["Staff Member Last Name"].isin(["Dhiman", "Kallen", "Williams","Baig","Leonard", "Tate","Desandre","Lamons"])

In [33]:
#mask_hs_data_df

In [34]:
final_hs_data_df = hs_data_df[mask_hs_data_df]

In [35]:
#final_hs_data_df

In [36]:
final_cap_update= pd.merge(survey_filtered_df, final_hs_data_df, on=("Email Address"), how = "inner")

In [37]:
final_cap_update["diff_days"] =final_cap_update["Date_x"]- final_cap_update["Date_y"] #if i remove "date" from the merge, then we can look at the day difference between the two times they met


In [38]:
#final_cap_update.head()

In [39]:
cap_survey_final_draft_df = final_cap_update.query('diff_days.dt.days <3') #removing entries where the completion diff is more than three days = which removes most of the anomalies

In [40]:
#cap_survey_final_draft_df

In [41]:
final_draft = cap_survey_final_draft_df.drop(columns = ["Student First Name","Student Last Name", "Date_y", "diff_days"]) #removing unneeded columns - could do this in code earlier when i merged

In [42]:
#final_draft

In [43]:
#final_draft.dtypes

In [45]:
final_draft.to_csv('W:\\hs database\\Final.csv') #writes to hs database folder as a csv